In [28]:
import sys
import os.path as op
import numpy as np
import pandas as pd
import scipy.stats as stats
import statsmodels.api as sm
import matplotlib as mpl
import matplotlib.pyplot as plt
import seaborn as sns

sys.path.append(op.join(op.expanduser("~"), "code", "style"))
from colors import get_colors

co, palettes = get_colors()

from general.basic.config import get_plot_defaults, set_rcparams

mpl.rcParams = set_rcparams(mpl.rcParams)
mpl.rcParams["axes.grid"] = False
d = get_plot_defaults()
co = d.get("colors", None)
colws = d.get("colws", None)
font = d.get("font", None)
lws = d.get("lws", None)
pad = d.get("pad", None)
palettes = d.get("palettes", None)

In [5]:
# import CSV
ssheet_dir = "/Users/dschonhaut/Box/projects/centiloid_calibration/ssheets"
data = pd.read_csv(op.join(ssheet_dir, "final_cohort_SUVR_w_CL_DS.csv"))

print(f"data: {data.shape}")

data: (234, 6)


In [43]:
mu_ycstd = 1.013595674
mu_adstd = 2.087774945
mu_diff = mu_adstd - mu_ycstd

print(f"mu_diff: {mu_diff}")

mu_diff: 1.074179271


In [21]:
stats.linregress(x=data["SUVR_LBL"], y=data["SUVR_klunk"])

LinregressResult(slope=1.0138985032663592, intercept=-0.06888052028495029, rvalue=0.9755443810506123, pvalue=1.2049777487662909e-154, stderr=0.01499809784234462, intercept_stderr=0.029211562151494966)

In [22]:
slope, icpt, corr, *_ = stats.linregress(x=data["SUVR_LBL"], y=data["SUVR_klunk"])
slope2, icpt2, corr2, *_ = stats.linregress(x=data["SUVR_klunk"], y=data["SUVR_LBL"])

print(
    f"SUVR_Std ~ SUVR_NStd: slope = {slope:.6f}, icpt = {icpt:.6f}, r^2 = {corr**2:.6f}"
)
print(
    f"SUVR_NStd ~ SUVR_Std: slope = {slope2:.6f}, icpt = {icpt2:.6f}, r^2 = {corr2**2:.6f}"
)

SUVR_Std ~ SUVR_NStd: slope = 1.013899, icpt = -0.068881, r^2 = 0.951687
SUVR_NStd ~ SUVR_Std: slope = 0.938641, icpt = 0.155814, r^2 = 0.951687


In [19]:
def rmse(y, yhat):
    """Return the root mean square error between y and yhat"""
    return np.sqrt(np.mean((y - yhat) ** 2))

In [48]:
print(
    sm.OLS(endog=data["SUVR_klunk"], exog=pib_calc).fit().summary(),
    end="\n\n" + ("*" * 94) + "\n\n",
)
print(sm.OLS(endog=data["SUVR_klunk"], exog=pib_calc2).fit().summary())

                                 OLS Regression Results                                
Dep. Variable:             SUVR_klunk   R-squared (uncentered):                   0.997
Model:                            OLS   Adj. R-squared (uncentered):              0.997
Method:                 Least Squares   F-statistic:                          6.968e+04
Date:                Mon, 01 Apr 2024   Prob (F-statistic):                   1.34e-290
Time:                        16:14:03   Log-Likelihood:                          183.75
No. Observations:                 234   AIC:                                     -365.5
Df Residuals:                     233   BIC:                                     -362.0
Df Model:                           1                                                  
Covariance Type:            nonrobust                                                  
                 coef    std err          t      P>|t|      [0.025      0.975]
-----------------------------------------

In [37]:
# Fit linear regression in statsmodels
mod = sm.OLS(endog=data["SUVR_klunk"], exog=data["SUVR_LBL"]).fit()
mod2 = sm.OLS(endog=data["SUVR_LBL"], exog=data["SUVR_klunk"]).fit()

print(mod.summary())
print(f"rsquared: {mod.rsquared}, {mod2.rsquared}")
print(f"adj rsquared: {mod.rsquared_adj}, {mod2.rsquared_adj}")
print(f"LLF: {mod.llf}, {mod2.llf}")
print(f"AIC: {mod.aic}, {mod2.aic}")

pd.DataFrame(np.vstack((mod.resid, mod2.resid)).T, columns=["resid", "resid2"])

                                 OLS Regression Results                                
Dep. Variable:             SUVR_klunk   R-squared (uncentered):                   0.997
Model:                            OLS   Adj. R-squared (uncentered):              0.997
Method:                 Least Squares   F-statistic:                          6.804e+04
Date:                Mon, 01 Apr 2024   Prob (F-statistic):                   2.11e-289
Time:                        14:44:11   Log-Likelihood:                          180.98
No. Observations:                 234   AIC:                                     -360.0
Df Residuals:                     233   BIC:                                     -356.5
Df Model:                           1                                                  
Covariance Type:            nonrobust                                                  
                 coef    std err          t      P>|t|      [0.025      0.975]
-----------------------------------------

,resid,resid2
0,-0.041246,0.045740
1,-0.012685,0.016939
2,0.304012,-0.305650
3,0.028925,-0.025653
4,-0.062442,0.067362
...,...,...
229,-0.010481,0.017096
230,0.014392,-0.006654
231,-0.085617,0.094739
232,-0.062138,0.068315


In [16]:
pib_calc = (data["SUVR_LBL"] * slope) + icpt
pib_calc2 = (data["SUVR_LBL"] - icpt2) / slope2

pd.DataFrame(np.vstack((pib_calc, pib_calc2)).T, columns=["pib_calc", "pib_calc2"])

,pib_calc,pib_calc2
0,1.054418,1.014324
1,1.129852,1.093588
2,1.007271,0.964785
3,1.051984,1.011767
4,1.071755,1.032542
...,...,...
229,1.842622,1.842543
230,2.304149,2.327499
231,2.201137,2.219258
232,1.446999,1.426835


In [41]:
np.mean(data["SUVR_CL_Daniel"]), np.mean(data["SUVR_CL_Klunk"]), rmse(
    data["SUVR_CL_Daniel"], data["SUVR_CL_Klunk"]
)

(77.36480046643929, 77.36480046643929, 2.3156512012005854)

In [50]:
gamma2, 100 * ((icpt2 - (slope2 * mu_ycstd)) / (slope2 * mu_diff))

(-109.87323335630374, -78.90644174297448)

In [13]:
# Calculate CL conversion
beta = 100 * (slope / mu_diff)
gamma = 100 * ((icpt - mu_ycstd) / mu_diff)
data["SUVR_CL_Daniel"] = (data["SUVR_LBL"] * beta) + gamma

beta2 = 100 / (slope2 * mu_diff)
gamma2 = 100 * (-(icpt2 / (slope2 * mu_diff)) - (mu_ycstd / mu_diff))
data["SUVR_CL_Klunk"] = (data["SUVR_LBL"] * beta2) + gamma2
# data["SUVR_CL_Klunk"] = 100 * ((((data["SUVR_klunk"] - icpt2) / slope2) - mu_ycstd) / mu_diff)
# beta2 = 100 * (1 / slope2)
# gamma2 = ()
data

,LBLID,SUVR_LBL,SUVR_klunk,Dx,SUVR_estimated_from_LBL,CL_estimated,SUVR_CL_Daniel,SUVR_CL_Klunk
0,B09-270,1.1079,1.044095,YC,1.014383,0.072944,3.801940,0.067462
1,B10-201,1.1823,1.145541,YC,1.093650,7.456233,10.828212,7.450427
2,B10-202,1.0614,1.343800,YC,0.964841,-4.541612,-0.589479,-4.546892
3,B10-203,1.1055,1.111915,YC,1.011826,-0.165227,3.575286,-0.170699
4,B10-204,1.1250,1.039650,YC,1.032602,1.769909,5.416849,1.764353
...,...,...,...,...,...,...,...,...
229,B19-263,1.8853,1.836431,AD,1.842638,77.220377,77.218920,77.211515
230,B19-266,2.3405,2.307235,AD,2.327616,122.393405,120.207612,122.382563
231,B19-285,2.2389,2.107694,AD,2.219369,112.310849,110.612597,112.300448
232,B19-288,1.4951,1.402519,AD,1.426912,38.497804,40.368772,38.490638
